In [1]:
import os
os.chdir('../')

from ast import arg
from DeepMTP.main import DeepMTP
from DeepMTP.simple_hyperband import BaseWorker
from DeepMTP.simple_hyperband import HyperBand
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC
from DeepMTP.utils.data_utils import data_process, BaseDataset
from DeepMTP.utils.utils import generate_config
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

In [2]:
# define the configuration space
cs= CS.ConfigurationSpace()

lr= CSH.UniformFloatHyperparameter(
    "learning_rate", lower=1e-6, upper=1e-3, default_value="1e-3", log=True
)
cs.add_hyperparameters([lr])

embedding_size= CSH.UniformIntegerHyperparameter(
    "embedding_size", lower=8, upper=2048, default_value=64, log=False
)

instance_branch_layers= CSH.UniformIntegerHyperparameter(
    "instance_branch_layers", lower=1, upper=2, default_value=1, log=False
)

instance_branch_nodes_per_layer= CSH.UniformIntegerHyperparameter(
    "instance_branch_nodes_per_layer", lower=8, upper=2048, default_value=64, log=False
)

target_branch_layers = CSH.UniformIntegerHyperparameter(
    "target_branch_layers", lower=1, upper=2, default_value=1, log=False
)

target_branch_nodes_per_layer = CSH.UniformIntegerHyperparameter(
    "target_branch_nodes_per_layer", lower=8, upper=2048, default_value=64, log=False
)

dropout_rate = CSH.UniformFloatHyperparameter(
    "dropout_rate", lower=0.0, upper=0.9, default_value=0.4, log=False
)

batch_norm = CSH.CategoricalHyperparameter("batch_norm", [True, False])

cs.add_hyperparameters(
    [
        embedding_size,
        instance_branch_layers,
        instance_branch_nodes_per_layer,
        target_branch_layers,
        target_branch_nodes_per_layer,
        dropout_rate,
        batch_norm,
    ]
)

cond = CS.GreaterThanCondition(dropout_rate, instance_branch_layers, 1)
cond2 = CS.GreaterThanCondition(batch_norm, instance_branch_layers, 1)
cond3 = CS.GreaterThanCondition(dropout_rate, target_branch_layers, 1)
cond4 = CS.GreaterThanCondition(batch_norm, target_branch_layers, 1)

cs.add_condition(CS.OrConjunction(cond, cond3))
cs.add_condition(CS.OrConjunction(cond2, cond4))

(batch_norm | instance_branch_layers > 1 || batch_norm | target_branch_layers > 1)

In [3]:
data = load_process_MLC(dataset_name='yeast', variant='undivided')
train, val, test, data_info = data_process(data, validation_setting='B', verbose=True)

Processing...
yeast:undivided - exists, not redownloading
Done
Interaction file: 2d numpy array format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: binary

-- Test set was not provided, could not detect if novel instances exist or not 
-- Test set was not provided, could not detect if novel targets exist or not 

Instance features file: processing features... Done

Cross input consistency for (numpy) interaction data and instance features checks out
-- Same instance ids in the interaction and features files for the train set

Splitting train to train-test according to validation setting B... Done
Splitting train to train-val according to validation setting B... Done


In [4]:
data_info

{'detected_validation_setting': 'B',
 'detected_problem_mode': 'classification',
 'instance_branch_input_dim': 103,
 'target_branch_input_dim': 14}

In [5]:
config = {    

    'hpo_results_path': './hyperband/',

    'instance_branch_input_dim': data_info['instance_branch_input_dim'],
    'target_branch_input_dim': data_info['target_branch_input_dim'],
    'validation_setting': data_info['detected_validation_setting'],
    'enable_dot_product_version': True,
    'problem_mode': data_info['detected_problem_mode'],

    'compute_mode': 'cuda:1',
    'train_batchsize': 512,
    'val_batchsize': 512,
    'num_epochs': 6,
    'num_workers': 8,

    'metrics': ['hamming_loss', 'auroc'],
    'metrics_average': ['macro'],
    'patience': 10,

    'evaluate_train': True,
    'evaluate_val': True,

    'verbose': True,
    'results_verbose': False,
    'use_early_stopping': True,
    'use_tensorboard_logger': True,
    'wandb_project_name': 'DeepMTP_v2',
    'wandb_project_entity': 'diliadis',
    'metric_to_optimize_early_stopping': 'loss',
    'metric_to_optimize_best_epoch_selection': 'loss',

    'instance_branch_architecture': 'MLP',

    'target_branch_architecture': 'MLP',

    'save_model': True,

    'eval_every_n_epochs': 10,

    'additional_info': {'eta': 3, 'max_budget': 9}
    }

In [6]:
worker = BaseWorker(
    train, val, test, data_info, config, 'loss'
)

In [7]:
worker.base_config

{'hpo_results_path': './hyperband/',
 'instance_branch_input_dim': 103,
 'target_branch_input_dim': 14,
 'validation_setting': 'B',
 'enable_dot_product_version': True,
 'problem_mode': 'classification',
 'compute_mode': 'cuda:1',
 'train_batchsize': 512,
 'val_batchsize': 512,
 'num_epochs': 6,
 'num_workers': 8,
 'metrics': ['hamming_loss', 'auroc'],
 'metrics_average': ['macro'],
 'patience': 10,
 'evaluate_train': True,
 'evaluate_val': True,
 'verbose': True,
 'results_verbose': False,
 'use_early_stopping': True,
 'use_tensorboard_logger': True,
 'wandb_project_name': 'DeepMTP_v2',
 'wandb_project_entity': 'diliadis',
 'metric_to_optimize_early_stopping': 'loss',
 'metric_to_optimize_best_epoch_selection': 'loss',
 'instance_branch_architecture': 'MLP',
 'target_branch_architecture': 'MLP',
 'save_model': True,
 'eval_every_n_epochs': 10,
 'additional_info': {'eta': 3, 'max_budget': 9}}

In [8]:
hb = HyperBand(
    base_worker=worker,
    configspace=cs,
    eta=config['additional_info']['eta'],
    max_budget=config['additional_info']['max_budget'],
    direction="min",
    verbose=True
)

These are the pre-calculate brackets and successive halving runs:
{9: {'n_i': [9, 3, 1], 'r_i': [1, 3, 9], 'num_iters': 3}, 3: {'n_i': [3], 'r_i': [9], 'num_iters': 1}}


In [9]:
best_overall_config = hb.run_optimizer()

-- Running bracket with starting budget: 9
---- Evaluating configuration... 
Selected device: cuda:1


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Starting training...


wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Starting testing... Calculating test performance... Done
Done


...

test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▆▅▄▃▃▂▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,▁▁
val_loss,█▆▅▄▃▃▂▁▁
test_auroc_macro,0.50301
test_hamming_loss_macro,0.69599
train_auroc_macro,0.51922


+-------+--------+--------+--------------------+-------------+
|  mode | #epoch |  loss  | hamming_loss_macro | auroc_macro |
+-------+--------+--------+--------------------+-------------+
| train |   0    | 0.8353 |       0.622        |    0.5121   |
| train |   1    | 0.8137 |         -          |      -      |
| train |   2    | 0.796  |         -          |      -      |
| train |   3    | 0.7815 |         -          |      -      |
| train |   4    | 0.7694 |         -          |      -      |
| train |   5    | 0.7592 |         -          |      -      |
| train |   6    | 0.7506 |         -          |      -      |
| train |   7    | 0.7433 |         -          |      -      |
| train |   8    | 0.7368 |       0.3028       |    0.5192   |
+-------+--------+--------+--------------------+-------------+
+------+--------+--------+--------------------+-------------+----------------+
| mode | #epoch |  loss  | hamming_loss_macro | auroc_macro | early_stopping |
+------+--------+------

Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 1
Epoch:3... Done
  Validating... Done
Epoch:4... Done
  Validating... Done
Epoch:5... Done
  Validating... Done
Epoch:6... Done
  Validating... Done
Epoch:7... Done
  Validating... Done
Epoch:8... Done
  Validating... Calculating val performance... Done
Done
Starting testing... Calculating test performance... Done
Done


test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▃▃▃▃▂▂▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,█▁
val_loss,█▄▄▄▃▂▂▁▁
test_auroc_macro,0.66655
test_hamming_loss_macro,0.21015
train_auroc_macro,0.72903


+-------+--------+--------+--------------------+-------------+
|  mode | #epoch |  loss  | hamming_loss_macro | auroc_macro |
+-------+--------+--------+--------------------+-------------+
| train |   0    | 0.6629 |       0.3028       |    0.5019   |
| train |   1    | 0.5045 |         -          |      -      |
| train |   2    | 0.4912 |         -          |      -      |
| train |   3    | 0.4887 |         -          |      -      |
| train |   4    | 0.4798 |         -          |      -      |
| train |   5    | 0.4672 |         -          |      -      |
| train |   6    | 0.4552 |         -          |      -      |
| train |   7    | 0.4419 |         -          |      -      |
| train |   8    | 0.4262 |       0.2077       |    0.729    |
+-------+--------+--------+--------------------+-------------+
+------+--------+--------+--------------------+-------------+----------------+
| mode | #epoch |  loss  | hamming_loss_macro | auroc_macro | early_stopping |
+------+--------+------

# Load the best model and generate predictions for the test set

In [10]:
best_model = DeepMTP(best_overall_config.info['config'], best_overall_config.info['model_dir'])

Loading checkpoint from ./hyperband/18_06_2022__21_43_27/18_06_2022__21_47_24/model.pt...  
Done
Selected device: cuda:1
Applying saved weights... Done


In [11]:
best_model_results = best_model.predict(test, verbose=True)

Calculating  performance... Done


In [12]:
best_model_results

({'hamming_loss_target_0': 0.2727272727272727,
  'auroc_target_0': 0.7462740134872212,
  'hamming_loss_target_1': 0.4049586776859504,
  'auroc_target_1': 0.6415290870267173,
  'hamming_loss_target_2': 0.27892561983471076,
  'auroc_target_2': 0.7797087585034014,
  'hamming_loss_target_3': 0.24380165289256198,
  'auroc_target_3': 0.8181342600377408,
  'hamming_loss_target_4': 0.23553719008264462,
  'auroc_target_4': 0.7077205882352942,
  'hamming_loss_target_5': 0.2644628099173554,
  'auroc_target_5': 0.6456913474051038,
  'hamming_loss_target_6': 0.15702479338842976,
  'auroc_target_6': 0.6681501547987616,
  'hamming_loss_target_7': 0.17148760330578514,
  'auroc_target_7': 0.609380164047712,
  'hamming_loss_target_8': 0.06198347107438017,
  'auroc_target_8': 0.6117474302496329,
  'hamming_loss_target_9': 0.11363636363636363,
  'auroc_target_9': 0.6164017800381437,
  'hamming_loss_target_10': 0.128099173553719,
  'auroc_target_10': 0.582135759058248,
  'hamming_loss_target_11': 0.2520661